# Cleaning a PostgreSQL Database
![Clean PostgreSQL Database](Project_Image.jpeg)

In this project, you will work with data from a hypothetical Super Store to challenge and enhance your SQL skills in data cleaning. This project will engage you in identifying top categories based on the highest profit margins and detecting missing values, utilizing your comprehensive knowledge of SQL concepts.

## Data Dictionary:

### `orders`:
| Column | Definition | Data type | Comments |
|--------|------------|-----------|----------|
| `row_id`| Unique Record ID | `INTEGER` |
| `order_id` | Identifier for each order in table | `TEXT` | Connects to `order_id` in `returned_orders` table |
| `order_date` | Date when order was placed | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |
| `region` | Region Customer belongs to | `TEXT` | Connects to `region` in `people` table |
| `product_id` | Identifier of Product bought | `TEXT` | Connects to `product_id` in `products` table |
| `sales` | Total Sales Amount for the Line Item | `DOUBLE PRECISION` |
| `quantity` | Total Quantity for the Line Item | `DOUBLE PRECISION` |
| `discount` | Discount applied for the Line Item | `DOUBLE PRECISION` |
| `profit` | Total Profit earned on the Line Item | `DOUBLE PRECISION` |

### `returned_orders`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `returned`| Yes values for Order / Line Item Returned | `TEXT` |
| `order_id` | Identifier for each order in table | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |

### `people`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `person`| Name of Salesperson credited with Order | `TEXT` |
| `region` | Region Salesperson in operating in | `TEXT` |

### `products`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `product_id`| Unique Identifier for the Product | `TEXT` |
| `category` | Category Product belongs to | `TEXT` |
| `sub_category` | Sub Category Product belongs to | `TEXT` |
| `product_name` | Detailed Name of the Product | `TEXT` |

As you can see in the Data Dictionary above, date fields have been written to the `orders` table as `TEXT` and numeric fields like sales, profit, etc. have been written to the `orders` table as `Double Precision`. You will need to take care of these types in some of the queries. This project is an excellent opportunity to apply your SQL skills in a practical setting and gain valuable experience in data cleaning and analysis. Good luck, and happy querying!

# Final Solution
-----

In [32]:
-- top_five_products_each_category
SELECT *
FROM 
(SELECT 
	p.category,
	p.product_name,
	ROUND(CAST(SUM(o.sales) AS NUMERIC),2) AS product_total_sales,
	ROUND (CAST(SUM(o.profit) AS NUMERIC),2) AS product_total_profit,
	RANK() OVER (PARTITION BY p.category ORDER BY SUM(o.sales) DESC)AS product_rank
FROM orders AS o
LEFT JOIN products AS p
ON o.product_id = p.product_id
GROUP BY p.category,p.product_name
) AS temp
WHERE product_rank < 6
;


,category,product_name,product_total_sales,product_total_profit,product_rank
0,Furniture,"Hon Executive Leather Armchair, Adjustable",58193.48,5997.25,1
1,Furniture,"Office Star Executive Leather Armchair, Adjust...",51449.80,4925.80,2
2,Furniture,"Harbour Creations Executive Leather Armchair, ...",50121.52,10427.33,3
3,Furniture,"SAFCO Executive Leather Armchair, Black",41923.53,7154.28,4
4,Furniture,"Novimex Executive Leather Armchair, Adjustable",40585.13,5562.35,5
5,Office Supplies,"Eldon File Cart, Single Width",39873.23,5571.26,1
6,Office Supplies,"Hoover Stove, White",32842.60,-2180.63,2
7,Office Supplies,"Hoover Stove, Red",32644.13,11651.68,3
8,Office Supplies,"Rogers File Cart, Single Width",29558.82,2368.82,4
9,Office Supplies,"Smead Lockers, Industrial",28991.66,3630.44,5


In [33]:
SELECT *
FROM orders

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,postal_code,market,region,product_id,sales,quantity,discount,profit,shipping_cost,order_priority
0,957,MX-2014-105921,2014-05-28,2014-06-03,Standard Class,ZC-21910,Zuschuss Carroll,Consumer,San Salvador,San Salvador,El Salvador,NaN,LATAM,Central,TEC-AC-10004626,342.080,2.0,0.00,0.0000,21.713,Medium
1,24359,ID-2013-61442,2013-01-15,2013-01-21,Standard Class,JB-16000,Joy Bell-,Consumer,Manila,National Capital,Philippines,NaN,APAC,Southeast Asia,OFF-BI-10001400,122.400,5.0,0.15,0.0000,21.710,Low
2,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,United States,10024.0,US,East,TEC-AC-10003033,2309.650,7.0,0.00,762.1845,933.570,Critical
3,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,Australia,NaN,APAC,Oceania,FUR-CH-10003950,3709.395,9.0,0.10,-288.7650,923.630,Critical
4,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,Australia,NaN,APAC,Oceania,TEC-PH-10004664,5175.171,9.0,0.10,919.9710,915.490,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,29002,IN-2014-62366,2014-06-19,2014-06-19,Same Day,KE-16420,Katrina Edelman,Corporate,Kure,Hiroshima,Japan,NaN,APAC,North Asia,OFF-FA-10000746,65.100,5.0,0.00,4.5000,0.010,Medium
51286,35398,US-2014-102288,2014-06-20,2014-06-24,Standard Class,ZC-21910,Zuschuss Carroll,Consumer,Houston,Texas,United States,77095.0,US,Central,OFF-AP-10002906,0.444,1.0,0.80,-1.1100,0.010,Medium
51287,40470,US-2013-155768,2013-12-02,2013-12-02,Same Day,LB-16795,Laurel Beltran,Home Office,Oxnard,California,United States,93030.0,US,West,OFF-EN-10001219,22.920,3.0,0.00,11.2308,0.010,High
51288,9596,MX-2012-140767,2012-02-18,2012-02-22,Standard Class,RB-19795,Ross Baird,Home Office,Valinhos,São Paulo,Brazil,NaN,LATAM,South,OFF-BI-10000806,13.440,2.0,0.00,2.4000,0.003,Medium


In [34]:
-- impute_missing_values
WITH missing AS
(SELECT *
FROM orders 
WHERE quantity IS NULL ),
	
unit_prices AS
(
SELECT product_id, 
	SUM(sales) AS total_sales,
	SUM(quantity) AS total_quantity,
	CAST(SUM(sales) / SUM(quantity) AS NUMERIC) AS unit_price
FROM orders 
WHERE quantity IS NOT NULL
	AND quantity != 0
GROUP BY product_id
)

SELECT DISTINCT m.product_id,
	m.discount,
	m.market,
	m.region,
	m.sales,
	m.quantity,
	ROUND(CAST((m.sales / u.unit_price) AS NUMERIC ),0) AS calculated_quantity
FROM missing AS m
LEFT JOIN unit_prices as u
ON m.product_id = u.product_id

		

,product_id,discount,market,region,sales,quantity,calculated_quantity
0,FUR-ADV-10000571,0.00,EMEA,EMEA,438.960,NaN,4
1,FUR-ADV-10004395,0.00,EMEA,EMEA,84.120,NaN,4
2,FUR-BO-10001337,0.15,US,West,308.499,NaN,3
3,TEC-STA-10003330,0.00,Africa,Africa,506.640,NaN,2
4,TEC-STA-10004542,0.00,Africa,Africa,160.320,NaN,4


# Data Exploration 
----

# Notes
## To Do List
1. Find top 5 products from each category with highest sales
   - Sorted by category in ascending order and
   - Sorted by sales in descending order
2. Impute missing values in quantity colum by determining the unit price of each product

In [35]:
SELECT *
FROM
(SELECT 
	p.category,  p.product_name,
	RANK() OVER (PARTITION BY category ORDER BY SUM(o.sales)) AS product_rank
FROM orders AS o
LEFT JOIN products AS p
ON o.product_id = p.product_id
GROUP BY p.category,p.product_name, o.sales) as subq
WHERE subq.category = 'Technology'

,category,product_name,product_rank
0,Technology,Maxell 4.7GB DVD-R 5/Pack,1
1,Technology,Maxell 4.7GB DVD-R 5/Pack,2
2,Technology,"Cisco Speaker Phone, VoIP",3
3,Technology,Maxell 4.7GB DVD+R 5/Pack,3
4,Technology,Maxell 4.7GB DVD-R 5/Pack,3
...,...,...,...
7368,Technology,Canon imageCLASS 2200 Advanced Copier,7369
7369,Technology,Canon imageCLASS 2200 Advanced Copier,7370
7370,Technology,Canon imageCLASS 2200 Advanced Copier,7371
7371,Technology,Canon imageCLASS 2200 Advanced Copier,7372


In [36]:
--- Test Area
WITH missing AS
(SELECT *
FROM orders 
WHERE quantity IS NULL ),
	
unit_prices AS
(
SELECT product_id, 
	SUM(sales) AS total_sales,
	SUM(quantity) AS total_quantity,
	CAST(SUM(sales) / SUM(quantity) AS NUMERIC) AS unit_price
FROM orders 
WHERE quantity IS NOT NULL
	AND quantity != 0
GROUP BY product_id
)

SELECT DISTINCT m.product_id,
	m.discount,
	m.market,
	m.region,
	m.sales,
	u.unit_price,
	ROUND(CAST((m.sales / u.unit_price) AS NUMERIC ),0) AS calculated_quantity
FROM missing AS m
LEFT JOIN unit_prices as u
ON m.product_id = u.product_id




,product_id,discount,market,region,sales,unit_price,calculated_quantity
0,FUR-ADV-10000571,0.00,EMEA,EMEA,438.960,109.740000,4
1,FUR-ADV-10004395,0.00,EMEA,EMEA,84.120,19.628000,4
2,FUR-BO-10001337,0.15,US,West,308.499,106.462400,3
3,TEC-STA-10003330,0.00,Africa,Africa,506.640,229.936615,2
4,TEC-STA-10004542,0.00,Africa,Africa,160.320,38.814316,4


In [37]:
SELECT *
FROM orders 
WHERE quantity IS NULL

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,postal_code,market,region,product_id,sales,quantity,discount,profit,shipping_cost,order_priority
0,50688,IV-2013-8930,2013-09-04,2013-09-04,Same Day,MJ-7740,Max Jones,Consumer,Abidjan,Lagunes,Cote d'Ivoire,NaN,Africa,Africa,TEC-STA-10003330,506.640,NaN,0.00,217.800,65.54,Medium
1,43428,RS-2011-1760,2011-11-29,2011-12-04,Standard Class,TH-11235,Tiffany House,Corporate,Severodvinsk,Arkhangel'sk,Russia,NaN,EMEA,EMEA,FUR-ADV-10000571,438.960,NaN,0.00,140.400,53.77,High
2,34093,CA-2011-154599,2011-04-12,2011-04-17,Standard Class,KN-16450,Kean Nguyen,Corporate,Redondo Beach,California,United States,90278.0,US,West,FUR-BO-10001337,308.499,NaN,0.15,-18.147,24.38,Medium
3,46824,AG-2014-9060,2014-12-27,2015-01-01,Standard Class,YS-11880,Yana Sorensen,Corporate,Algiers,Alger,Algeria,NaN,Africa,Africa,TEC-STA-10004542,160.320,NaN,0.00,24.000,12.66,Medium
4,43722,RS-2011-3610,2011-12-08,2011-12-10,First Class,ND-8460,Neil Ducich,Corporate,Usol'ye-Sibirskoye,Irkutsk,Russia,NaN,EMEA,EMEA,FUR-ADV-10004395,84.120,NaN,0.00,42.060,11.83,Medium


In [38]:
SELECT product_id, 
	SUM(sales) AS total_sales,
	SUM(quantity) AS total_quantity,
	CAST(SUM(sales) / SUM(quantity) AS NUMERIC) AS unit_price
FROM orders 
WHERE quantity IS NOT NULL
	AND quantity != 0
GROUP BY product_id

,product_id,total_sales,total_quantity,unit_price
0,OFF-SME-10000950,4.032,1,4.032000
1,TEC-CIS-10001717,5691.888,11,517.444364
2,OFF-ST-10002905,427.200,24,17.800000
3,OFF-GLO-10000201,38.280,5,7.656000
4,OFF-PA-10003656,538.152,21,25.626286
...,...,...,...,...
10287,OFF-BI-10004738,142.050,13,10.926923
10288,TEC-PAN-10001172,484.080,11,44.007273
10289,FUR-BO-10004441,3434.980,17,202.057647
10290,OFF-AME-10000870,237.864,15,15.857600


In [39]:
SELECT product_id, 
	discount,
	market,
	region,
	sales,
	quantity,
	(sales / quantity) AS calculated_quantity
FROM orders 
WHERE quantity IS NULL

,product_id,discount,market,region,sales,quantity,calculated_quantity
0,TEC-STA-10003330,0.00,Africa,Africa,506.640,NaN,NaN
1,FUR-ADV-10000571,0.00,EMEA,EMEA,438.960,NaN,NaN
2,FUR-BO-10001337,0.15,US,West,308.499,NaN,NaN
3,TEC-STA-10004542,0.00,Africa,Africa,160.320,NaN,NaN
4,FUR-ADV-10004395,0.00,EMEA,EMEA,84.120,NaN,NaN


In [40]:
SELECT *
FROM 
(SELECT *
FROM orders AS o
LEFT JOIN products AS p
ON o.product_id = p.product_id
) temp
WHERE temp.quantity IS NULL

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,city,state,country,postal_code,market,region,product_id,sales,quantity,discount,profit,shipping_cost,order_priority,product_id,category,sub_category,product_name
0,50688,IV-2013-8930,2013-09-04,2013-09-04,Same Day,MJ-7740,Max Jones,Consumer,Abidjan,Lagunes,Cote d'Ivoire,NaN,Africa,Africa,TEC-STA-10003330,506.640,NaN,0.00,217.800,65.54,Medium,TEC-STA-10003330,Technology,Machines,"StarTech Printer, Red"
1,43428,RS-2011-1760,2011-11-29,2011-12-04,Standard Class,TH-11235,Tiffany House,Corporate,Severodvinsk,Arkhangel'sk,Russia,NaN,EMEA,EMEA,FUR-ADV-10000571,438.960,NaN,0.00,140.400,53.77,High,FUR-ADV-10000571,Furniture,Furnishings,"Advantus Frame, Erganomic"
2,34093,CA-2011-154599,2011-04-12,2011-04-17,Standard Class,KN-16450,Kean Nguyen,Corporate,Redondo Beach,California,United States,90278.0,US,West,FUR-BO-10001337,308.499,NaN,0.15,-18.147,24.38,Medium,FUR-BO-10001337,Furniture,Bookcases,O'Sullivan Living Dimensions 2-Shelf Bookcases
3,46824,AG-2014-9060,2014-12-27,2015-01-01,Standard Class,YS-11880,Yana Sorensen,Corporate,Algiers,Alger,Algeria,NaN,Africa,Africa,TEC-STA-10004542,160.320,NaN,0.00,24.000,12.66,Medium,TEC-STA-10004542,Technology,Machines,"StarTech Calculator, Durable"
4,43722,RS-2011-3610,2011-12-08,2011-12-10,First Class,ND-8460,Neil Ducich,Corporate,Usol'ye-Sibirskoye,Irkutsk,Russia,NaN,EMEA,EMEA,FUR-ADV-10004395,84.120,NaN,0.00,42.060,11.83,Medium,FUR-ADV-10004395,Furniture,Furnishings,"Advantus Door Stop, Durable"
